<a href="https://colab.research.google.com/github/AaronSanchezBelber/Capstone_Repository/blob/master/Binance_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install python-binance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os 

In [ ]:
os.environ['binance_api'] = 'OI0aiCyAYktPpILpNh5nJSVmtygqNnXMXcBVjSrI9lx8MkMv1c5WwruIBhGw5jia'
os.environ['binance_secret'] =  'A7AAsjWJ9tt8C1HBjISWbFdynvNgwLNHoSnJaUog1C7bYngaAr1tDRyG2jPXuWFP'

In [ ]:
from binance.client import Client
api_key = os.environ.get('binance_api')
api_secret = os.environ.get('binance_secret')

In [ ]:
client = Client(api_key, api_secret)

In [ ]:
'''Unfortunately, the python-binance library does not have support for the demo environment previously discussed.
 However, we were able to connect by manually changing the API endpoint URL in the library like this:'''

'Unfortunately, the python-binance library does not have support for the demo environment previously discussed.\n However, we were able to connect by manually changing the API endpoint URL in the library like this:'

In [ ]:
client.API_URL = 'https://testnet.binance.vision/api'

In [ ]:
# get balances for all assets & some account information
print(client.get_account())

{'makerCommission': 0, 'takerCommission': 0, 'buyerCommission': 0, 'sellerCommission': 0, 'canTrade': True, 'canWithdraw': False, 'canDeposit': False, 'brokered': False, 'updateTime': 1666788334186, 'accountType': 'SPOT', 'balances': [{'asset': 'BNB', 'free': '1000.00000000', 'locked': '0.00000000'}, {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'}, {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'}, {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'}, {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'}, {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'}, {'asset': 'USDT', 'free': '10000.00000000', 'locked': '0.00000000'}, {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}], 'permissions': ['SPOT']}


In [ ]:
# get latest price from Binance API
client.get_symbol_ticker(symbol="BTCUSDT")

{'symbol': 'BTCUSDT', 'price': '20793.37000000'}

The next step is to initialize the socket manager. When we do that, the library will create a new thread which we’ve started in the code snippet above. We need to pass through a symbol, which in this case is BTCUSDT. We also specify it to call our custom btc_trade_history function every time a new message comes in

In [ ]:
from time import sleep

from binance import BinanceSocketManager

In [ ]:
btc_price = {'error':False}

In [ ]:
def btc_trade_history(msg):
    ''' define how to process incoming WebSocket messages '''
    if msg['e'] != 'error':
      print(msg['c'])
      btc_price['last'] = msg['c']
      btc_price['bid'] = msg['b']
      btc_price['last'] = msg['a']
      btc_price['error'] = False
    else:
        btc_price['error'] = True

In [ ]:
client = Client(api_key, api_secret)
bs = BinanceSocketManager(client)
socket = bs.trade_socket('BTCUSDT')

In [ ]:
# Creador: https://www.youtube.com/watch?v=rc_Y6rdBqXM
x = 0
while x != 10:
  await socket.__aenter__()
  msg = await socket.recv()
  frame = createframe(msg)
  x = x + 1
  print(frame)

    symbol                    Time     Price
0  BTCUSDT 2022-10-26 14:34:00.873  20780.29
    symbol                    Time     Price
0  BTCUSDT 2022-10-26 14:34:00.890  20780.29
    symbol                    Time     Price
0  BTCUSDT 2022-10-26 14:34:00.909  20780.29
    symbol                    Time     Price
0  BTCUSDT 2022-10-26 14:34:00.909  20780.57
    symbol                    Time     Price
0  BTCUSDT 2022-10-26 14:34:00.935  20780.56
    symbol                    Time     Price
0  BTCUSDT 2022-10-26 14:34:00.962  20780.12
    symbol                    Time     Price
0  BTCUSDT 2022-10-26 14:34:00.964  20780.26
    symbol                    Time     Price
0  BTCUSDT 2022-10-26 14:34:00.974  20779.43
    symbol                    Time     Price
0  BTCUSDT 2022-10-26 14:34:01.044  20779.65
    symbol                    Time     Price
0  BTCUSDT 2022-10-26 14:34:01.080  20779.45


In [ ]:
import pandas as pd
def createframe(msg):
  df = pd.DataFrame([msg])
  df = df.loc[:,['s','E','p']]
  df.columns = ['symbol', 'Time', 'Price']
  df.Price = df.Price.astype(float)
  df.Time = pd.to_datetime(df.Time, unit = 'ms')
  return df

In [ ]:
createframe(msg)

,symbol,Time,Price
0,BTCUSDT,2022-10-26 14:15:13.021,20760.11


In [ ]:
# Create a sql and add certain information
# import sqlalchemy
# engine = sqlalchemy.create_engine('sqlite:///BTCUSDTstream.db')
frame.to_sql('BTCUSDT', engine)

In [ ]:
# subscribe to a stream
ws.start_symbol_ticker_socket(callback=btc_trade_history, symbol='BTCUSDT')
#Add another one
# bsm.start_symbol_ticker_socket(callback=btc_trade_history, symbol='ETHUSDT')

NameError: ignored

In [ ]:
# stop websocket
ws.stop()

In [ ]:
########HISTORICAL DATA###########
# valid intervals - 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M

# get timestamp of earliest date data is available
timestamp = client._get_earliest_valid_timestamp('BTCUSDT', '1d')
print(timestamp)

1664928000000


In [ ]:
# request historical candle (or klines) data
bars = client.get_historical_klines('BTCUSDT', '1d', timestamp, limit=1000)
print(bars)

[[1664928000000, '20087.49000000', '25000.00000000', '10000.00000000', '20158.20000000', '4568.13982700', 1665014399999, '91696644.27054668', 88593, '2838.23406400', '56975222.46145802', '0'], [1665014400000, '20158.45000000', '21050.94000000', '5045.53000000', '19962.39000000', '10220.49264700', 1665100799999, '205902493.11437532', 197665, '6283.14353600', '126627616.70047324', '0'], [1665100800000, '19962.39000000', '21182.55000000', '4031.00000000', '19530.59000000', '8620.59442700', 1665187199999, '170355357.58724846', 158407, '5364.24803500', '106039435.34037074', '0'], [1665187200000, '19530.28000000', '50000.00000000', '15000.00000000', '19417.64000000', '7651.95253500', 1665273599999, '149166687.81384724', 152866, '4907.22836400', '95683259.75513835', '0'], [1665273600000, '19417.91000000', '22266.00000000', '19192.25000000', '19441.55000000', '7220.22349700', 1665359999999, '140373161.84493830', 150097, '4507.66387400', '87633770.97962301', '0'], [1665360000000, '19441.5500000

In [ ]:
import csv
with open('btc_bars.csv', 'w', newline='') as f:
	wr = csv.writer(f)
	for line in bars:
		wr.writerow(line)